# Introduction

## What is image classification?
![Image classification](https://miro.medium.com/max/3840/1*oB3S5yHHhvougJkPXuc8og.gif)

## What are notebooks?
![Notebooks](https://threathunterplaybook.com/_images/JUPYTER_ARCHITECTURE.png)

## How are we going to tackle this example?
![CRISP-DM](https://miro.medium.com/max/494/1*VH9RlYPSjL6YOtBtHxunqQ.png)

-------------------------
-------------------------
# Business Understanding

## Objective

The imagery data comes from a variety of sources. The bulk of the data was collected in-field by CIMMYT and CIMMYT partners in Ethiopia and Tanzania. The remainder of the data is sourced from public images found on Google Images.

The aim of this challenge is to build a machine learning model to accurately classify the wheat in the images as: healthy, stem rust, or leaf rust.

Some images may contain both stem and leaf rust, there is always one type of rust that is more dominant than the other, i.e. you will not find images where both appear equally. The goal is to classify the image according to the type of wheat rust that appears most prominently in the image. The values for each classification can be between 0 and 1, inclusive, and should represent the probability that the wheat in the image belongs in each category.



## Strategy + problem simplification

![image classification](https://miro.medium.com/max/1128/1*ATIx1SmkEH0FaL_5fMvX2w.jpeg)

* Collect images (DONE)
* Label images (DONE)
* Define as an Image classification problem (!)


-------------------------
-------------------------
![CRISP-DM](https://www.mdpi.com/applsci/applsci-09-02407/article_deploy/html/images/applsci-09-02407-g001.png)
# Data understanding

## Where is our data?

![Notebooks](https://threathunterplaybook.com/_images/JUPYTER_ARCHITECTURE.png)

* Our data is currently in a server, managed by Kaggle and assigned to us.

```
/kaggle/input/cgiar-computer-vision-for-crop-disease
|-- /test/test
|-- /train/train
    |-- /healthy_wheat
    |-- /leaf_rust
    |-- /stem_rust

```

In [ ]:
class_path = "/kaggle/input/cgiar-computer-vision-for-crop-disease/train/train/healthy_wheat"
! ls {class_path}

In [ ]:
class_path = "/kaggle/input/cgiar-computer-vision-for-crop-disease/train/train/healthy_wheat"
! ls {class_path}

## How to read image files?

In python, we can find multiple libraries which will allow us to read and display images, some of them are:
- PIL
- opencv
- matplotlib

In [ ]:
import matplotlib.pyplot as plt

image = plt.imread(class_path+"/I9JOL9.jpg")
plt.imshow(image)
plt.show()

## What is an image?

* Images are 2D matrix composed of pixels.
* Pixels can have one or three values defining their intensity

![Images](https://mozanunal.com/images/pixel.png)

In [ ]:
# Images are matrix
image.shape

* RGB Images are basically a 3D matrix. or 3 2D matrix, one for each color intencity (Red Green and Blue).
![RGB images](https://webstyleguide.com/wsg2/graphics/graphics/7.07.gif)

In [ ]:
#each pixel is a color RGB
image_R = image.copy()
image_R[:,:,1]=0
image_R[:,:,2]=0
plt.imshow(image_R)
plt.show()

image_G = image.copy()
image_G[:,:,0]=0
image_G[:,:,2]=0
plt.imshow(image_G)
plt.show()

image_B = image.copy()
image_B[:,:,1]=0
image_B[:,:,0]=0
plt.imshow(image_B)
plt.show()

In [ ]:
# each pixel contains the intensity encoded as a number from 0 to 255

image_gray = image.mean(axis=2)

plt.imshow(image_gray, cmap="gray")
plt.show()

# we can get a subimage by indexing the axis
image_small = image_gray[::50,::50]

plt.imshow(image_small, cmap="gray")
plt.show()

# for each cell, we can also check the intensity value
image_small[:4,:4]

-------------------------
-------------------------
![CRISP-DM](https://www.mdpi.com/applsci/applsci-09-02407/article_deploy/html/images/applsci-09-02407-g001.png)
# Data Preparation

![data pipeline](https://i2.wp.com/mlexplained.com/wp-content/uploads/2018/02/%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%B3%E3%82%B7%E3%83%A7%E3%83%83%E3%83%88-2018-02-07-10.32.59.png?fit=1998%2C938&ssl=1)


## Datasets

In [ ]:
!ls /kaggle/input/cgiar-computer-vision-for-crop-disease/train/train

In [ ]:
import torchvision
import torch

base_dataset = torchvision.datasets.ImageFolder(
    root='/kaggle/input/cgiar-computer-vision-for-crop-disease/train/train',
    transform=None)

print(base_dataset.classes)
print(len(base_dataset))

In [ ]:
base_dataset.imgs

In [ ]:
image, label = base_dataset[0]

plt.imshow(image)
plt.show()

## Transforms

* Data augmentation is a commonly used technique in computer vision problems which helps to deal with small datasets and helps algorithms converge more smoothly.

![data augmentation](https://www.frontiersin.org/files/Articles/469305/fncom-13-00083-HTML/image_m/fncom-13-00083-g003.jpg)

In [ ]:
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.RandomRotation(20),
     transforms.RandomVerticalFlip()])#,
     #transforms.ToTensor(),
     #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
for i in range(5):
    plt.imshow(transform(image))
    plt.show()

## Dataloaders


![dataloaders](https://miro.medium.com/max/3480/1*3vAYjhGh_EopD0cRdxrbOQ.png)

In [ ]:
# transforms
transform = transforms.Compose(
    [transforms.Resize((224, 224)),#(224, 224)),
     transforms.RandomRotation(20),
     transforms.RandomVerticalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# dataset
base_dataset = torchvision.datasets.ImageFolder(
    root='/kaggle/input/cgiar-computer-vision-for-crop-disease/train/train',
    transform=transform)

# dataloader
base_loader = torch.utils.data.DataLoader(base_dataset, batch_size=16,shuffle=True, num_workers=4)

-------------------------
-------------------------
![CRISP-DM](https://www.mdpi.com/applsci/applsci-09-02407/article_deploy/html/images/applsci-09-02407-g001.png)
# Modeling

![modeling](https://drive.google.com/uc?id=1jnHXWqQdBAaJYmIlCNyRINX8KJgUghnM)


![resnet18](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSVhix4X-U2-fuTuEdTIWpkhFqTiCAoGROmxw&usqp=CAU)

In [ ]:
# where does the model do all the operations?
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# creating a model based on a preexisting architecture
from torchvision import models
import torch.nn as nn

model = models.resnet18(pretrained=False)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(base_dataset.classes))

model.to(device)
model

### Training

![gradient descent](https://thumbs.gfycat.com/AngryInconsequentialDiplodocus-size_restricted.gif)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
# train
for epoch in range(1):
    for i, data in enumerate(base_loader, 0):
        # get the inputs
        inputs, labels = data
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs.to(device))
        loss = criterion(outputs.to(device), labels.to(device))
        loss.backward()
        optimizer.step()

In [ ]:
import time

# create arrays to save metrics
t_start = time.time()
loss_arr=[]
acc_arr=[]
for epoch in range(1):  # loop over the dataset multiple times
    
    #initialize epoch metrics
    t_epoch = time.time()
    running_loss = 0.0
    running_acc = 0.0
    running_n = 0
    
    for i, data in enumerate(base_loader, 0):
        # get the inputs
        inputs, labels = data
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs.to(device))
        loss = criterion(outputs.to(device), labels.to(device))
        loss.backward()
        optimizer.step()
        
        # compute minibatch metrics
        running_acc += (torch.argmax(outputs, dim=1).to(device) == labels.to(device)).float().sum()
        running_n += len(labels)
        running_loss += loss.item()
        
    # compute epoch metrics
    running_acc = running_acc/running_n
    running_loss = running_loss/running_n
    loss_arr.append(running_loss)
    acc_arr.append(running_acc)
    
    # print epoch metrics
    print('epoch: {} loss: {} acc: {} time epoch: {} time total: {}'.format(epoch + 1, running_loss, running_acc, time.time()-t_epoch, time.time()-t_start))

print('Finished Training in {} seconds'.format(time.time()-t_start))

### Evaluate

In [ ]:
view_dataset = torchvision.datasets.ImageFolder(
    root='/kaggle/input/cgiar-computer-vision-for-crop-disease/train/train',
    transform=None)

transform_view = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

import numpy as np
indexes = np.random.choice(len(view_dataset), 5)
for i in indexes:
    x,y = view_dataset[i]
    x_view = transform_view(x)
    x_tr = transform(x)
    
    y_out = model(x_tr.to(device).unsqueeze(0))
    y_prob = nn.functional.softmax(y_out, dim=1)
    y_pred = torch.argmax(y_prob, dim=1)
    
    fig, axs = plt.subplots(1,2)
    axs[0].imshow(x)
    axs[0].set_title("[{}] label {}".format(i,y))
    
    axs[1].imshow(transforms.ToPILImage()(x_tr))
    axs[1].set_title("pred {}".format(y_pred[0]))
    plt.show()

### Train test split?

* current size of the dataset: 562

In [ ]:
len(base_dataset)

In [ ]:
# split our dataset
train_dataset, val_dataset = torch.utils.data.random_split(base_dataset, [450, 112])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16,shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16,shuffle=True, num_workers=4)

In [ ]:
# create arrays to save metrics
t_start = time.time()
loss_arr=[]
acc_arr=[]
val_acc_arr=[]
for epoch in range(50):  # loop over the dataset multiple times
    
    #initialize epoch metrics
    t_epoch = time.time()
    running_loss = 0.0
    running_acc = 0.0
    running_n = 0
    running_val_acc = 0.0
    running_val_n = 0
    
    # train
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs.to(device))
        loss = criterion(outputs.to(device), labels.to(device))
        loss.backward()
        optimizer.step()
        
        # compute minibatch metrics
        running_acc += (torch.argmax(outputs, dim=1).to(device) == labels.to(device)).float().sum()
        running_n += len(labels)
        running_loss += loss.item()
    
    # validate/evaluate
    for i, data in enumerate(val_loader, 0):
        # get the inputs
        inputs, labels = data

        # forward
        outputs = model(inputs.to(device))
        loss = criterion(outputs.to(device), labels.to(device))
        
        # compute minibatch metrics
        running_val_acc += (torch.argmax(outputs, dim=1).to(device) == labels.to(device)).float().sum()
        running_val_n += len(labels)
        
    # compute epoch metrics
    running_acc = running_acc/running_n
    running_val_acc = running_val_acc/running_val_n
    running_loss = running_loss/running_n
    loss_arr.append(running_loss)
    acc_arr.append(running_acc)
    val_acc_arr.append(running_val_acc)
    
    # print epoch metrics
    print('epoch: {} loss: {} acc: {} valacc: {} time epoch: {} time total: {}'.format(epoch + 1, running_loss, running_acc, running_val_acc, time.time()-t_epoch, time.time()-t_start))

print('Finished Training in {} seconds'.format(time.time()-t_start))

In [ ]:
plt.plot(val_acc_arr, label="val_acc")
plt.plot(acc_arr, label="acc")
plt.legend()
plt.show()
plt.plot(loss_arr, label="loss")
plt.legend()
plt.show()

# What to do now?

![CRISP-DM](https://www.mdpi.com/applsci/applsci-09-02407/article_deploy/html/images/applsci-09-02407-g001.png)

![CRISP-DM](https://miro.medium.com/max/494/1*VH9RlYPSjL6YOtBtHxunqQ.png)
